In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from pathlib import Path
from bs4 import BeautifulSoup
from urllib.parse import urljoin

import os
import requests
import time

# Configure Selenium options for running in Colab
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(options=options)

def search_and_download_duckduckgo(query, num=100):
    # URL for DDG image search
    url = f"https://duckduckgo.com/?q={query}&t=h_&iax=images&ia=images"

    # Loads webpage at specified URL
    driver.get(url)

    # Wait for the page to load
    time.sleep(5)

    # Parse the page source
    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Finds all images
    image_elements = soup.find_all('img', limit=num)

    # Setup path to a data folder
    image_dir = "data/animals"

    # Create new directory contingent on query
    os.makedirs(os.path.join(image_dir, query), exist_ok=True)

    page_num = 1
    while len(image_elements) < num:
        url = f"https://duckduckgo.com/?q={query}&t=h_&iax=images&ia=images&page={page_num}"
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        new_image_elements = soup.find_all('img', limit=num - len(image_elements))
        image_elements.extend(new_image_elements)
        page_num += 1

    print(f"\nFound {len(image_elements)} images of {query}")

    for i, img in enumerate(image_elements):
      # Extract the image URL via attributes
      image_url = img.get('src') or img.get('data-src')

      # Handle relative URLs by converting them to absolute URLs
      if image_url and not image_url.startswith('http'):
        image_url = urljoin(url, image_url)

      # Check if the URL is a direct link to the image
      if image_url and image_url.startswith("http"):
        # Get the image URL
        try:
          response = requests.get(image_url)

          # Renames image based on query & position
          filename = f"{query}_{i}.jpg"

          # Saves image to proper directory
          with open(os.path.join(image_dir, query, filename), 'wb') as f:
              f.write(response.content)
          print(f"Downloaded {filename}")
        except Exception as e:
          print(f"Error downloading {image_url}: {e}")
      else:
        print(f"Skipping non-direct URL: {image_url}")

In [3]:
search_and_download_duckduckgo("cats", 250)
search_and_download_duckduckgo("dogs", 250)
search_and_download_duckduckgo("birds", 250)


Found 250 images of cats
Downloaded cats_0.jpg
Downloaded cats_1.jpg
Downloaded cats_2.jpg
Downloaded cats_3.jpg
Downloaded cats_4.jpg
Downloaded cats_5.jpg
Downloaded cats_6.jpg
Downloaded cats_7.jpg
Downloaded cats_8.jpg
Downloaded cats_9.jpg
Downloaded cats_10.jpg
Downloaded cats_11.jpg
Downloaded cats_12.jpg
Downloaded cats_13.jpg
Downloaded cats_14.jpg
Downloaded cats_15.jpg
Downloaded cats_16.jpg
Downloaded cats_17.jpg
Downloaded cats_18.jpg
Downloaded cats_19.jpg
Downloaded cats_20.jpg
Downloaded cats_21.jpg
Downloaded cats_22.jpg
Downloaded cats_23.jpg
Downloaded cats_24.jpg
Downloaded cats_25.jpg
Downloaded cats_26.jpg
Downloaded cats_27.jpg
Downloaded cats_28.jpg
Downloaded cats_29.jpg
Downloaded cats_30.jpg
Downloaded cats_31.jpg
Downloaded cats_32.jpg
Downloaded cats_33.jpg
Downloaded cats_34.jpg
Downloaded cats_35.jpg
Downloaded cats_36.jpg
Downloaded cats_37.jpg
Downloaded cats_38.jpg
Downloaded cats_39.jpg
Downloaded cats_40.jpg
Downloaded cats_41.jpg
Downloaded cats_42

In [4]:
# Set path of animals folder
data_dir = Path("data/")
image_dir = data_dir / "animals"

# Print path, # of sub-folders
image_dir, len(list(image_dir.iterdir()))

(PosixPath('data/animals'), 3)

In [5]:
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

device = "cuda" if torch.cuda.is_available() else "cpu"

# Ensure reproducibility
torch.manual_seed(42)

# Augment data (assists CNN in learning minute details)
data_transforms = transforms.Compose([
    # Resize images to 64x64 for TinyVGG compatibility (basis of CNN)
    transforms.Resize(size=(64, 64)),

    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(),

    transforms.RandomResizedCrop(64, scale=(.08, 1)),

    # Turn the image into a torch.Tensor
    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the images from subfolders
imageDataset = datasets.ImageFolder(root=image_dir, transform=data_transforms)

# 80/20 split between train & validation
train_size = int(0.8 * len(imageDataset))
validation_size = len(imageDataset) - train_size

# Randomly split the data
train_dataset, validation_dataset = random_split(imageDataset, [train_size, validation_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

In [6]:
device

'cuda'

In [7]:
imageDataset[0][0].shape

torch.Size([3, 64, 64])

In [8]:
# Verify lengths of dataset
len(train_dataset), len(validation_dataset)

(600, 150)

In [9]:
# Verify classes of folders in list & dict form
imageDataset.classes, imageDataset.class_to_idx

(['birds', 'cats', 'dogs'], {'birds': 0, 'cats': 1, 'dogs': 2})

In [10]:
# Building a CNN
class animalIdentifier(nn.Module):
  """
  Model architecture to determine animal
  Structure is as follows: For the two conv blocks: conv -> ReLU -> conv -> ReLU -> MaxPool
  MaxPool is especially important as it decreases the spatial size of an image,
  reducing the parameters & computation of the network. Essentially it allows for higher
  higher level of pattern recognition in images (i.e. from edges to parts to objects & onward)
  """

  def __init__(self, input_shape: int, hidden_units: int, output_shape: int):
    # Instantiate NN
    super().__init__()

    # First conv block
    self.conv_block_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    # Second conv block
    self.conv_block_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    # Third conv block
    self.conv_block_3 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )

    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Dropout(p=0.5),  # 50% dropout
        nn.Linear(in_features=hidden_units * 8 * 8, # Exact spatial dimension calculated
                  out_features=output_shape)
    )

  def forward(self, x):
    return self.classifier(self.conv_block_3(self.conv_block_2(self.conv_block_1(x))))

In [11]:
# Instantiate the model with, in order: # of color channels, hidden_units, # of classes
animalModel = animalIdentifier(input_shape=3,
                               hidden_units=64,
                               output_shape=len(imageDataset.classes)).to(device)

animalModel

animalIdentifier(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Drop

In [12]:
# Retrieve a single batch of data (batch of images & corresponding labels)
image_batch, label_batch = next(iter(train_loader))
image_batch.shape, label_batch.shape

(torch.Size([32, 3, 64, 64]), torch.Size([32]))

In [13]:
# Try a forward pass
animalModel(image_batch.to(device))

tensor([[-0.0030,  0.0080, -0.0064],
        [-0.0074, -0.0038, -0.0150],
        [-0.0296,  0.0148, -0.0243],
        [-0.0006,  0.0199, -0.0140],
        [-0.0251, -0.0015, -0.0044],
        [ 0.0006,  0.0045, -0.0094],
        [-0.0246,  0.0297, -0.0178],
        [-0.0105, -0.0008, -0.0255],
        [-0.0020,  0.0072, -0.0092],
        [ 0.0025,  0.0504, -0.0254],
        [-0.0412, -0.0056, -0.0283],
        [-0.0136,  0.0135, -0.0259],
        [-0.0050,  0.0137, -0.0130],
        [-0.0010,  0.0281, -0.0204],
        [-0.0308,  0.0106, -0.0154],
        [-0.0118,  0.0142, -0.0116],
        [-0.0151,  0.0074, -0.0137],
        [-0.0070,  0.0142, -0.0284],
        [-0.0246,  0.0291,  0.0116],
        [-0.0134,  0.0362, -0.0200],
        [-0.0253,  0.0003, -0.0210],
        [-0.0215,  0.0272, -0.0185],
        [-0.0164,  0.0290, -0.0158],
        [-0.0068,  0.0070, -0.0001],
        [ 0.0065,  0.0045, -0.0167],
        [ 0.0090, -0.0032, -0.0087],
        [ 0.0021,  0.0299, -0.0321],
 

In [14]:
# Train step
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer,
               device=device):

  # Put the model in train mode
  model.train()

  train_loss, train_acc = 0, 0

  # Loop through batches
  for batch in dataloader:
    # Extract values from batch
    images, labels = batch

    # Send to device
    images, labels = images.to(device), labels.to(device)

    # Forward pass
    predictions = model(images)

    # Calculate loss
    loss = loss_fn(predictions, labels) # Logits
    train_loss += loss.item() # Cumulative loss

    # Calculate accuracy
    prediction_classes = torch.argmax(predictions, dim=1) # Predicted class (logits to classes)
    train_acc += (prediction_classes == labels).sum().item() # Sum the number of correct predictions

    # Set gradients to zero
    optimizer.zero_grad()

    # Backpropagation
    loss.backward()

    # Step (learning rate)
    optimizer.step()

  # Average loss & accuracy per batch
  train_loss = train_loss / len(dataloader)
  train_acc = train_acc / len(dataloader.dataset) # Total correct / total samples

  return train_loss, train_acc

In [15]:
# Validation step
def validation_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module,
              device=device):

  # Put the model in eval mode
  model.eval()

  validation_loss, validation_acc = 0, 0

  # Disables gradient computation, reduces memory usage & increases speed
  with torch.inference_mode():

    # Loop through batches
    for batch in dataloader:
      # Extract values from batch
      images, labels = batch

      # Send to device
      images, labels = images.to(device), labels.to(device)

      # Forward pass
      predictions = model(images)

      # Calculate the loss
      loss = loss_fn(predictions, labels)
      validation_loss += loss.item()

      # Calculate accuracy
      prediction_classes = torch.argmax(predictions, dim=1) # Predicted class (logits to classes)
      validation_acc += (prediction_classes == labels).sum().item() # Sum the number of correct predictions

  # Average loss & accuracy per batch
  validation_loss = validation_loss / len(dataloader)
  validation_acc = validation_acc / len(dataloader.dataset)

  return validation_loss, validation_acc

In [16]:
# Combine both train_step() & validation_step() into one function
from tqdm.auto import tqdm

# Takes in train & validation dataloaders, as well as everything needed to compute
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          validation_dataloader: torch.utils.data.DataLoader,
          optimizer: torch.optim.Optimizer,
          loss_fn: torch.nn.Module,
          epochs: int = 5,
          device=device):

  # Create dictionary to store results
  results = {"train_loss": [],
             "train_acc": [],
             "validation_loss": [],
             "validation_acc": []}

  # Loop through train & validation steps for # of epochs
  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)

    validation_loss, validation_acc = validation_step(model=model,
                                    dataloader=validation_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    # Print data per epoch
    print(f"Epoch: {epoch} | Train loss: {train_loss:.4f} | Train acc: {train_acc:.4f} | Validation loss: {validation_loss:.4f} | Validation acc: {validation_acc:.4f}")

    # Update results dictionary
    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["validation_loss"].append(validation_loss)
    results["validation_acc"].append(validation_acc)

  # Return filled results
  return results

In [17]:
# Time to train

NUM_EPOCHS = 125

# Recreate model from above
animalModelV2 = animalIdentifier(input_shape=3,
                               hidden_units=64,
                               output_shape=len(imageDataset.classes)).to(device)

# Configure loss function & optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=animalModelV2.parameters(),
                            lr=0.001)

# Timer
from timeit import default_timer as timer
start_time = timer()

# Train
results = train(model=animalModelV2,
                train_dataloader=train_loader,
                validation_dataloader=validation_loader,
                optimizer=optimizer,
                loss_fn=loss_fn,
                epochs=NUM_EPOCHS,
                device=device)

# End the timer
end_time = timer()
print(f"Total training time: {end_time - start_time:.3f} seconds")

  0%|          | 0/125 [00:00<?, ?it/s]

Epoch: 0 | Train loss: 1.0863 | Train acc: 0.3917 | Validation loss: 1.0395 | Validation acc: 0.3867
Epoch: 1 | Train loss: 0.9468 | Train acc: 0.5317 | Validation loss: 0.9451 | Validation acc: 0.5200
Epoch: 2 | Train loss: 0.9209 | Train acc: 0.5283 | Validation loss: 0.8658 | Validation acc: 0.5533
Epoch: 3 | Train loss: 0.8753 | Train acc: 0.5667 | Validation loss: 0.9074 | Validation acc: 0.6133
Epoch: 4 | Train loss: 0.7955 | Train acc: 0.6200 | Validation loss: 0.8958 | Validation acc: 0.5733
Epoch: 5 | Train loss: 0.7972 | Train acc: 0.6033 | Validation loss: 0.8408 | Validation acc: 0.5267
Epoch: 6 | Train loss: 0.8186 | Train acc: 0.5483 | Validation loss: 0.8399 | Validation acc: 0.5333
Epoch: 7 | Train loss: 0.7792 | Train acc: 0.6200 | Validation loss: 0.7864 | Validation acc: 0.6200
Epoch: 8 | Train loss: 0.7356 | Train acc: 0.6333 | Validation loss: 0.7694 | Validation acc: 0.6333
Epoch: 9 | Train loss: 0.7183 | Train acc: 0.6450 | Validation loss: 0.7130 | Validation ac

In [18]:
# Create model directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

# Create model name & save path
MODEL_NAME = "animalClassifier.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save model's weights to created path
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=animalModelV2.state_dict(),
           f=MODEL_SAVE_PATH)

Saving model to: models/animalClassifier.pth


In [19]:
# Initialize the model
peopleClassifier = animalIdentifier(input_shape=3,
                               hidden_units=64,
                               output_shape=len(imageDataset.classes))

# Load pre-trained weights
modelWeightsPath = MODEL_SAVE_PATH
stateDict = torch.load(modelWeightsPath)
peopleClassifier.load_state_dict(stateDict)

# Move to device
peopleClassifier.to(device)

<ipython-input-19-2c4f0e55305b>:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  stateDict = torch.load(modelWeightsPath)


animalIdentifier(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Drop

In [20]:
from google.colab import drive
print(drive.mount('/content/drive'))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
None


In [21]:
# Set path of animals folder
drive_dir = Path("drive/")
image_dir = drive_dir / "MyDrive/Hateocracy"

# Print path, # of sub-folders
image_dir, len(list(image_dir.iterdir()))

(PosixPath('drive/MyDrive/Hateocracy'), 10)

In [217]:
# Ensure reproducibility
torch.manual_seed(42)

# Augment data (assists CNN in learning minute details)
new_data_transforms = transforms.Compose([
    # Resize images to 64x64 for TinyVGG compatibility (basis of CNN)
    transforms.Resize(size=(64, 64)),

    # Flip the images randomly on the horizontal
    transforms.RandomHorizontalFlip(p=0.5),

    # Flip the images randomly on the vertical
    transforms.RandomVerticalFlip(p=0.5),

    transforms.RandomRotation(15),

    transforms.RandomResizedCrop(64, scale=(.08, 1)),

    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.2),

    # Turn the image into a torch.Tensor
    transforms.ToTensor(),

    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

# Load the images from subfolders
newImageDataset = datasets.ImageFolder(root=image_dir, transform=new_data_transforms)

# Create DataLoaders
new_train_loader = DataLoader(newImageDataset, batch_size=2, shuffle=True)

In [218]:
from sklearn.model_selection import KFold
from torch.utils.data import Subset

k_folds = 5

# Prepare cross-validation
kfold = KFold(n_splits=k_folds, shuffle=True)

for fold, (train_idx, val_idx) in enumerate(kfold.split(newImageDataset)):
    print(f'Fold {fold+1}/{k_folds}')

    # Prepare data loaders for this fold
    train_subset = Subset(newImageDataset, train_idx)
    new_train_loader = DataLoader(train_subset, batch_size=2, shuffle=True)

Fold 1/5
Fold 2/5
Fold 3/5
Fold 4/5
Fold 5/5


In [219]:
# Verify lengths of dataset
len(newImageDataset), len(newImageDataset.classes)

(109, 10)

In [220]:
# Modify final layer of model to match the number of classes in new dataset
num_classes = len(newImageDataset.classes)

# Load the pretrained model
peopleClassifier.classifier = nn.Sequential(
    nn.Flatten(),
    nn.Dropout(p=0.7),
    nn.Linear(in_features=64 * 8 * 8, out_features=num_classes)
)

# Move to device
peopleClassifier.to(device)

animalIdentifier(
  (conv_block_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_2): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv_block_3): Sequential(
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Drop

In [221]:
# Freeze layers to preserve learned features from pretrained model

for param in peopleClassifier.conv_block_1.parameters():
  param.requires_grad = False
for param in peopleClassifier.conv_block_2.parameters():
  param.requires_grad = False
for param in peopleClassifier.conv_block_3.parameters():
  param.requires_grad = False

for param in peopleClassifier.classifier.parameters():
  param.requires_grad = True

In [222]:
# Time to train
from torch.optim.lr_scheduler import ReduceLROnPlateau

NUM_EPOCHS = 125

# Reinitialize loss funciton & optimizer
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=peopleClassifier.parameters(),
                             lr=0.0001,
                             weight_decay=1e-5)

scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

# Timer
from timeit import default_timer as timer
start_time = timer()

for epoch in range(NUM_EPOCHS):
  newTrainLoss, newTrainAcc = train_step(peopleClassifier,
                                         new_train_loader,
                                         loss_fn,
                                         optimizer,
                                         device)

  print(f"Epoch: {epoch} | Train loss: {newTrainLoss:.4f} | Train acc: {newTrainAcc:.4f}")

  # Unfreeze each block every 25 epochs for further adaptation to dataset, thus decreasing loss
  if epoch == 41:
    for param in peopleClassifier.conv_block_3.parameters():
      param.requires_grad = True

    # Reinitialize optimizer to include newly unfrozen parameters
    optimizer = torch.optim.Adam(params=peopleClassifier.parameters(), lr=0.0001, weight_decay=1e-5)

  if epoch == 82:
    for param in peopleClassifier.conv_block_2.parameters():
      param.requires_grad = True

    # Reinitialize optimizer to include newly unfrozen parameters
    optimizer = torch.optim.Adam(params=peopleClassifier.parameters(), lr=0.0001, weight_decay=1e-5)

    # Once second conv block is unfrozen, step the scheduler after each epoch based on the training loss
    scheduler.step(newTrainLoss)

# End the timer
end_time = timer()
print(f"Total training time: {end_time - start_time:.3f} seconds")

Epoch: 0 | Train loss: 2.4486 | Train acc: 0.1250
Epoch: 1 | Train loss: 2.2674 | Train acc: 0.2273
Epoch: 2 | Train loss: 2.0986 | Train acc: 0.2614
Epoch: 3 | Train loss: 1.9372 | Train acc: 0.2614
Epoch: 4 | Train loss: 1.9074 | Train acc: 0.3636
Epoch: 5 | Train loss: 1.7583 | Train acc: 0.3636
Epoch: 6 | Train loss: 1.6735 | Train acc: 0.4205
Epoch: 7 | Train loss: 1.7110 | Train acc: 0.3636
Epoch: 8 | Train loss: 1.8043 | Train acc: 0.3977
Epoch: 9 | Train loss: 1.6771 | Train acc: 0.3750
Epoch: 10 | Train loss: 1.5405 | Train acc: 0.4205
Epoch: 11 | Train loss: 1.5666 | Train acc: 0.4432
Epoch: 12 | Train loss: 1.6245 | Train acc: 0.4091
Epoch: 13 | Train loss: 1.5382 | Train acc: 0.4545
Epoch: 14 | Train loss: 1.4322 | Train acc: 0.5114
Epoch: 15 | Train loss: 1.4365 | Train acc: 0.4659
Epoch: 16 | Train loss: 1.4044 | Train acc: 0.5114
Epoch: 17 | Train loss: 1.3429 | Train acc: 0.5341
Epoch: 18 | Train loss: 1.3188 | Train acc: 0.5568
Epoch: 19 | Train loss: 1.4736 | Train ac

In [224]:
# Create model directory
MODEL_PATH = Path("models")
MODEL_PATH.mkdir(parents=True,
                 exist_ok=True)

# Create model name & save path
MODEL_NAME = "peopleClassifier.pth"
MODEL_SAVE_PATH = MODEL_PATH / MODEL_NAME

# Save model's weights to created path
print(f"Saving model to: {MODEL_SAVE_PATH}")
torch.save(obj=animalModelV2.state_dict(),
           f=MODEL_SAVE_PATH)

Saving model to: models/peopleClassifier.pth
